In [19]:
import perceval as pcvl
import numpy as np
import matplotlib.pyplot as plt
import itertools
from Functions import realLrz
import matplotlib.pyplot as plt
import scipy as sc
from scipy import signal
from tabulate import tabulate
from dataRead import xpRead

ImportError: cannot import name 'realLrz' from 'Functions' (/Users/simsim/Documents/HIVER2023/Perceval/polyquantique/SandboxSimon/Results/Functions.py)

In [15]:
time, env1, env2,delay, table = xpRead('./acquired_data/xp_03/data.json')

extime = np.linspace(time[0],time[-1],10000)

env1Interp = np.interp(extime,time,env1)
env2Interp = np.interp(extime,time,env2)

In [16]:
laserPulse = realLrz(extime,10)

NameError: name 'realLrz' is not defined

In [ ]:


new_base,coeffsMGS = modified_Schmidt(waves,x)
C2MGS = coeffsMGS[1,1]